# 1. Parameter Study Using the Grid Iterator
## Introduction to QUEENS
QUEENS is a versatile software framework offering a wide range of cutting-edge algorithms for deterministic and probabilistic analyses, including parameter studies, sensitivity analysis, surrogate modeling, uncertainty quantification, and Bayesian inverse analysis. Built with a modular architecture, QUEENS enables efficient parallel queries of large-scale computational models, robust handling of data and resources, seamless switching between analysis types, and smooth scalability from laptops to high-performance computing clusters. To learn more, visit the QUEENS [website](https://www.queens-py.org), explore the source code on [GitHub](https://github.com/queens-py), or check out the [documentation](https://queens-py.github.io/queens).

### Overview
Over the course of the four tutorials you will learn
* how to use QUEENS: the general structure of a QUEENS script, a short Python script used to define and run experiments with QUEENS
* how to conduct various analysis types with QUEENS, including parameter studies, optimization, and uncertainty quantification
* how to run analyses with different models from analytical test functions to advanced numerical solvers like the 4C multiphysics framework

## Tutorial: analysis of the Rosenbrock function

In this tutorial, you will learn how to translate your planned multi-query analysis into a QUEENS experiment in the form of a Python script.
The structure of these scripts is always similar, where the common feature of a multi-query analysis is that you want to evaluate a single computational model
at many different input locations.
Thus, the main ingredients you need to define for your experiment are:
* the model
* the analysis method (defines the input points where to evaluate the model)
* the compute resource to evaluate the model

### Content
The example analyses we want to conduct in this tutorial is the following:
1. visualise the Rosenbrock function.
2. find the minimum of the Rosenbrock function.

___
#### **Task:** Run the following code cell.

You don't need to understand the code but you should run it once in order to setup the logging feature of QUEENS correctly for Jupyter notebooks.

In [ ]:
# Suppress excessive logging output
import logging
import os
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# logging.basicConfig(level=logging.INFO, format='%(message)s')
os.environ["DASK_DISTRIBUTED__LOGGING__DISTRIBUTED"] = "CRITICAL"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger("distributed").setLevel(level=logging.CRITICAL)


def visualize_grid_and_surface(X1, X2, Z, min_point=None):
    """Visualize grid and surface with optional minimum point.

    Parameters
    _________
    X1, X2 : 2D arrays
        Meshgrid arrays for x and y.
    Z : 2D array
        Function values f(X1, X2).
    min_point : tuple (x, y, z) or None
        If provided, highlights this point with a big red cross
        in both plots.
    """
    fig = make_subplots(
        rows=1,
        cols=2,
        specs=[[{"type": "scene"}, {"type": "scene"}]],
        subplot_titles=("3D Scatter colored by z", "Surface + grid points (z=0 plane)"),
    )

    # ___ Left: 3D scatter (share colorscale, no colorbar) ___
    fig.add_trace(
        go.Scatter3d(
            x=X1.flatten(),
            y=X2.flatten(),
            z=Z.flatten(),
            mode="markers",
            marker=dict(
                size=3,
                color=Z.flatten(),
                colorscale="Viridis",
                showscale=False,  # no duplicate colorbar
            ),
            showlegend=False,
        ),
        row=1,
        col=1,
    )

    # ___ Right: Surface (with single colorbar) ___
    fig.add_trace(
        go.Surface(
            x=X1,
            y=X2,
            z=Z,
            colorscale="Viridis",
            colorbar=dict(
                title="f(x1,x2)",
                x=1.05,  # push colorbar outside plot area
                len=0.75,  # shorten to avoid overlapping legend
            ),
        ),
        row=1,
        col=2,
    )

    # ___ Right: black grid points on x-y plane (z=0) ___
    fig.add_trace(
        go.Scatter3d(
            x=X1.flatten(),
            y=X2.flatten(),
            z=np.zeros_like(Z).flatten(),
            mode="markers",
            marker=dict(size=2, color="black"),
            showlegend=False,
        ),
        row=1,
        col=2,
    )

    # ___ Highlight user-provided minimum point ___
    if min_point is not None:
        min_x, min_y, min_z = min_point
        for col in [1, 2]:
            fig.add_trace(
                go.Scatter3d(
                    x=[min_x],
                    y=[min_y],
                    z=[min_z],
                    mode="markers",
                    marker=dict(size=5, color="red"),
                    name="Minimum",
                ),
                row=1,
                col=col,
            )

    # Layout / axes
    fig.update_layout(
        title="3D Scatter vs. Surface with Grid Projection",
        height=500,
        legend=dict(
            x=0.95, y=0.9, bgcolor="rgba(255,255,255,0.6)"  # move legend away from colorbar
        ),
        scene=dict(xaxis_title="x1", yaxis_title="x2", zaxis_title="f(x1,x2)", aspectmode="cube"),
        scene2=dict(xaxis_title="x1", yaxis_title="x2", zaxis_title="f(x1,x2)", aspectmode="cube"),
    )

    fig.show()

## Model Setup

We begin by setting up our model.
Here, we want to implement the (two-dimensional) Rosenbrock function.

The **Rosenbrock function** is a classic test problem in optimization, defined as

$
f(x_1, x_2) = (a - x_1)^2 + b \, (x_2 - x_1^2)^2 ,
$

where typically its parameters are
$
a = 1, \quad b = 100 .
$

As we will see this function is non-convex and features a narrow, curved valley, making it a challenging benchmark for optimization algorithms.

### **Task:** Implement the Rosenbrock function as a Python function that takes two arguments $x_1$ and $x_2$.

In [ ]:
def rosenbrock(x1, x2):
    a = 1
    b = 100
    f = (a - x1) **2 + b* (x2 - x1 **2)**2
    return f

## Visualising is a multi-query analysis task

To visualise the Rosenbrock function as a 3D surface, we need to evaluate it on a grid (or mesh) of points.
This is a *multi-query* scenario because creating a surface plot requires computing the function value at many different input locations across the domain.
With such an analytical model, this is straightforward—and you may already be familiar with doing this.
Nevertheless, let’s break it down step by step to see clearly how the process works.

### Generate mesh points with NumPy

To generate a regular mesh in NumPy,
 you can use `numpy.linspace` to create evenly spaced points along each axis and then pass these arrays to `numpy.meshgrid`,
 which combines them into two 2D arrays representing all coordinate pairs.
This allows you to build a full rectangular grid of points, which can be used to evaluate functions across the domain.

### **Task:** Create evenly spaced points
Create $10$ points per parameter on the following intervall $x_1 \in [-2.0, 2.0]$ and $ x_2 \in [-3.0, 3.0]$.

> For example, `np.linspace(-2, 2, 5)` gives five points between −2 and 2.

In [ ]:
import numpy as np

# Create grid
x1 = np.linspace(-2.0, 2.0, 10)
x2 = np.linspace(-3.0, 3.0, 10)
X1, X2 = np.meshgrid(x1, x2)

## Evaluate the Model

So far, we have:
1. Defined our model (the Rosenbrock function).
2. Built a grid of input points $(x_1, x_2)$ using `numpy.linspace` and `numpy.meshgrid`.

The next step is to evaluate the model at every point of the grid.
This means: for each coordinate pair $(x_1, x_2)$, we want to compute the function value $f(x_1, x_2)$.

For our function, we can directly exploit the **vectorization feature of NumPy** and run the following code.
In short: this line evaluates the Rosenbrock function at all grid points in one shot, turning our mesh of input coordinates into a mesh of output values.


In [ ]:
# Evaluate function on grid
Z = rosenbrock(X1, X2)

## Visualise the Rosenbrock function
The following code block introduces a function to visualise the grid points as well as the function values of the Rosenbrock function.
We introduce the function in a separate file `visualize_grid_and_surface.py` to keep this notebook lean. 
We will reuse the visualisation function several times, but you don't need to go through the lines in detail.

Running the next cell, you will see:
* On the **left**, a scatter plot of all grid points in 3D space, colored by their function values.
* On the **right**, the characteristic *banana-shaped* valley of the Rosenbrock function shown as a smooth surface, with the grid points projected onto the base plane for reference.

In [ ]:
visualize_grid_and_surface(X1, X2, Z)

## Congratulations, you have finished your first multi-query analysis!

The direct NumPy approach works perfectly for simple analytical functions, but it quickly becomes impractical for complex computational models—such as those involving Finite-Element solvers—where each evaluation is computationally expensive. In these cases, vectorized evaluation is no longer possible. Instead, we view the task as an embarrassingly parallel problem: each input point can be evaluated independently of the others.

This is exactly where QUEENS comes into play. It provides the parallelization and workflow management needed to move from simple Python functions to large-scale simulations. Under the hood, QUEENS applies the same core principle you just saw with NumPy—evaluating the model at many input points—but in a way that is scalable, robust, and designed for demanding computational models.

## Visualise the Rosenbrock function with QUEENS
We will now redo the multi-query task of visualising the Rosenbrock function using **QUEENS**.
This helps you learn the structure of a QUEENS experiment on a very simple setup.
The structure will remain the same later on, e.g., when we find the minimum of the Rosenbrock function with QUEENS.
For a simple analytic model, this may feel like overhead,
but it pays off as soon as you switch to another analysis method with the same model (a key feature of QUEENS) or when the model becomes more complex and computationally demanding.

Note that the approach shown here works with any Python function that encodes your model of interest.
If you already have functions from your research or application, it’s straightforward to make them work with QUEENS: write a thin wrapper that exposes exactly the input parameters you want to vary. For example, you might keep a general Rosenbrock implementation with parameters `a` and `b`, and provide a small wrapper that fixes `a` and `b` while varying only `x1` and `x2`. This pattern generalizes to more sophisticated models, enabling you to reuse the same QUEENS experiment structure across different analyses.

### Global settings for a QUEENS experiment: name and output directory

Every QUEENS experiment starts by defining the **name of the experiment** (which automatically created files and directories will be named after) and **output directory** (where the QUEENS output file will be placed).

#### **Task**: define a variable `experiment_name` to label your QUEENS run.
This name will be used automatically for generated output files and folders, so choosing something sensible makes it much easier to identify your results later.

> Important general note: if another experiment with the same name already exists, existing data will potentially be overwritten. To avoid this, you can change the experiment name.

Your experiment name should follow a few simple rules (similar to naming files or directories in Linux):
* Use a short, descriptive string that helps you remember what the experiment did.
* Avoid spaces — use underscores _ instead.
* Do not begin with a number.
* Do not use special characters.
* Keep it concise but meaningful.

##### Examples

##### Good names
* grid_iterator_rosenbrock
* rosenbrock_grid
* optimize_rosenbrock
* grid_test_x1x2
* sensitivity_analysis_demo

##### Bad names
* 1st_experiment → starts with a number
* grid iterator rosenbrock → contains spaces
* this_is_a_very_long_and_confusing_name_for_experiment → too long
* test!rosenbrock → contains special characters

#### Output directory
You don’t need to change the `output_dir` variable here — by default, we will write results into the current directory of this notebook.

If you do decide to set a different path, make sure the directory already exists, as QUEENS requires the output directory to be created beforehand.

#### Global Settings
In the code block, we also create a `GlobalSettings` object, which gathers the general information about the QUEENS experiment.
This ensures that the correct values (such as experiment name, output directory, and debug settings) are consistently used throughout the workflow.
Later on, `GlobalSettings` will also act as a Python context manager, making sure that everything is properly set up and cleaned up when running the experiment.


In [ ]:
from queens.global_settings import GlobalSettings

# Define name of QUEENS experiment and directory for output
experiment_name = "grid_iterator_rosenbrock"
output_dir = "./"

# Global settings
global_settings = GlobalSettings(
    experiment_name=experiment_name, output_dir=output_dir, debug=False
)

## QUEENS Model Setup

We can reuse our implementation of the Rosenbrock function from above.
However, QUEENS needs additional information about the input parameters $x_1, x_2$.
In particular, QUEENS must know:

- **The name** of the parameter (e.g., `x1`, `x2`)
- **The type** of the parameter (deterministic variable, random variable, or random field)
- **Its dimensionality** (scalar or vector-valued)
- **Its distribution** (e.g., uniform, normal) and associated properties (bounds, mean, variance, etc.)
- ...

Only with this information can QUEENS properly treat the parameters, generate samples, and propagate them through the Rosenbrock function.

### Model Parameters

Again, we restrict the two parameters to certain regions:

- $x_1 \in [-2.0, 2.0]$
- $x_2 \in [-3.0, 3.0]$

Such restrictions can be expressed in **QUEENS** by using the `Uniform` parameter type, which has a lower and an upper bound.

#### **Task**: Create a Uniform object for the input $ x_2 \in [-3.0, 3.0]$
> Hint: the first parameter is already defined, you only need to add the second one.

Finally, all parameter definitions are collected into a `Parameters` object. This container keeps track of all variable namesx and their properties.
Most importantly, it allows creating samples from the input space according to its properties.

#### **Task**: Add the second input $x_2$ as a keyword argument to the `Parameters` object.

The first parameter `x1` has already been defined for you.
Now we add `x2` to the Parameters object.

> Important: Make sure to use the correct variable name as the keyword, in this case `x2` (see definition of the Rosenbrock Python function).
> Specifically, if your function has the following signature:
 ```python
 def f(x1, x2, my_parameter, ...):
    ...
 ```
> then you must define the parameters object like this:
 ```python
Parameters(x1=..., x2=..., my_parameter=..., ...)
 ```


In [ ]:
from queens.distributions import Uniform
from queens.parameters import Parameters

# Model parameters
x1 = Uniform(lower_bound=-2.0, upper_bound=2.0)
x2 = Uniform(lower_bound=-3.0, upper_bound=3.0)
parameters = Parameters(x1=x1, x2=x2)

At this point, the `parameters` object contains both $x_1$ and $x_2$,
each with their respective domains, and is ready to be used for building the QUEENS model.

In order to finalize the QUEENS model, we still require multiple things:
1. A `scheduler`. It requests the compute resource and manages the execution of your model on that compute.

    For this tutorial, we choose a `Local` dask scheduler with a single worker. So your model is evaluated on the local machine. With the `num_jobs` parameter, you can choose the number of parallel model evaluations.

2. A `driver`. This object coordinates the evaluation of the forward model itself.

    The `Function` driver can be used to evaluate any Python function with QUEENS.
    Note that it takes both the `parameters` object as well as the callable Python function `rosenbrock` as inputs.
    In setting up the `parameters` and `function` it is important the arguments have the same keywords.
    In our case `parameters` was created with `x1` and `x2` keyword arguments and the `rosenbrock` functione expects exactly two argument with the same name.
    > Background: we are exploiting that we can always call a positional argument as keyword arguments in Python.

3. And finally, a QUEENS `model`, which takes both a `driver` (model evaluation routine) and a `scheduler` (compute resource).

    The `Simulation` model is the standard type of model in QUEENS.
    The fact that it takes both a `scheduler` and a `driver` reflects that in QUEENS a model always combines both the instruction of how to evaluate (`driver`) as well as the compute resources to actually execute and manage the evaluation (`scheduler`).

### **Task**: Run the following code cell to define a QUEENS model for the subsequent analyses.

In [ ]:
from queens.drivers import Function
from queens.models.simulation import Simulation
from queens.schedulers import Local

#### Model setup ####
scheduler = Local(global_settings.experiment_name, num_jobs=1)
driver = Function(parameters=parameters, function=rosenbrock)
model = Simulation(scheduler, driver)

### Analysis method: Grid (visualizing Rosenbrock as a 3D surface)

To visualize the Rosenbrock function, we will evaluate it on a **grid** of points inside the domain specified by our `Parameters` object.
In **QUEENS**, an **iterator** is responsible for generating the input points for multi-query analyses.
Here we will use the `Grid` iterator, which lays out points on a mesh over the ranges of the parameters.
It can generate rectilinear grids on both linear and logarithmic scales. 

**Key idea:**
- The **iterator** chooses *where* to evaluate the model.

#### Defining the grid design for the `Grid` iterator

The `Grid` iterator in QUEENS requires a **grid layout/resolution** for each parameter:
- **`num_grid_points`**: how many grid points to generate in this dimension  
- **`axis_type`**: type of axis spacing (e.g., `"lin"` for linear, `"log10"` for logarithmic)  
- **`data_type`**: the parameter data type (e.g., `FLOAT`)  

For example:
```python
{"x1": {"num_grid_points": 5, "axis_type": "lin", "data_type": "FLOAT"}}
```
means: generate 5 grid points, linearly spaced, for a float-valued parameter x1.

___
#### **Task**: Extend the grid design to include x2
We want to evaluate x2 at 10 points on a linear scale, also as a FLOAT.

In [ ]:
grid_design = {
        "x1": {"num_grid_points": 10, "axis_type": "lin", "data_type": "FLOAT"},
        "x2": {"num_grid_points": 10, "axis_type": "lin", "data_type": "FLOAT"},
    }

#### Define the iterator object

Every iterator in QUEENS requires some **common arguments** regardless of its type:

- **`model`**: the QUEENS model to work with (here, the Rosenbrock model)  
- **`parameters`**: the `Parameters` object defining the variable input parameters  
- **`global_settings`**: general settings of the experiment, including the experiment name and output directory  
- **`result_description`**: whether to write result files and what details to include  

Each iterator type can also have **unique properties**.  
For the **`Grid` iterator**, the key unique property is the **`grid_design`** dictionary, which specifies the resolution and layout per parameter as described and defined above.


In [ ]:
from queens.iterators.grid import Grid
# The method of the analysis is defined by the iterator type:
grid = Grid(
    grid_design=grid_design,
    model=model,
    parameters=parameters,
    global_settings=global_settings,
    result_description={"write_results": True},
)

### Running the experiment

Now that we have fully defined our `Grid` iterator, we can finally **run the experiment**.  
In QUEENS, we always execute experiments using the **`run_iterator`** function, wrapped inside the **`global_settings`** context manager.  
This ensures that all outputs (results, plots, logs) are properly handled according to the experiment configuration.

___

#### **Task**: Run your QUEENS experiment

> Note: You might have to restart the Jupyter kernel to rerun the experiment!

In [ ]:
from queens.main import run_iterator
from queens.utils.io import load_result

with global_settings:
    #### Analysis ####
    run_iterator(grid, global_settings=global_settings)

## QUEENS output

If activated in the result description, QUEENS output is written to a file called `<experiment_name>.pickle` in the directory defined by the argument `output_dir` of the global settings object: `<output_dir>/<experiment_name>.pickle`.
In the output folder, you also get a log file that contains the console output. It is called `<experiment_name>.log`.

In the case of the grid iterator, you get a dictionary including the raw input-output data and some statistics.

In [ ]:
#### Load Results ####
result_file = global_settings.result_file(".pickle")
results = load_result(result_file)
results

### Postprocessing

You can now interact with these results as with any other Python object.
For example, you can plot the results.

Here, we use the same plotting function as before. But in order to do so we first reshape the QUEENS input-output-data into the same structure that we saw in the initial minimal example.



In [ ]:
input_data = results["input_data"]
output_data = results["raw_output_data"]["result"]

X1_QUEENS = input_data[:,0].reshape(grid_design["x2"]["num_grid_points"],grid_design["x1"]["num_grid_points"])
X2_QUEENS = input_data[:,1].reshape(grid_design["x2"]["num_grid_points"],grid_design["x1"]["num_grid_points"])

Z_QUEENS = output_data.reshape(grid_design["x2"]["num_grid_points"],grid_design["x1"]["num_grid_points"])

visualize_grid_and_surface(X1_QUEENS,X2_QUEENS, Z_QUEENS)

### Validating QUEENS results against manual mesh generation

As you can see in the plot above, we obtain **visually indistinguishable**.  
This happens because the **generation of grid points under the hood inside the `Grid` iterator** is implemented in the same way as in our **minimal NumPy example** (i.e., using `numpy.linspace` to generate linearly spaced points before using `numpy.meshgrid`).

To check that the grid generated by QUEENS matches your own manually constructed mesh (using `numpy.linspace` / `numpy.meshgrid`), you can use `numpy.allclose`.  

This function verifies that two arrays are numerically close to each other, up to floating-point tolerances.

In [ ]:
print(f"X1 and X1_QUEENS are identical: {np.allclose(X1, X1_QUEENS)}")
print(f"X2 and X2_QUEENS are identical: {np.allclose(X2, X2_QUEENS)}")
print(f"Z and Z_QUEENS are identical: {np.allclose(Z, Z_QUEENS)}")

Thus, both approaches (`np.meshgrid` vs. `Grid` iterator) yield the same grid coordinates.

This confirms that the QUEENS `Grid` iterator behaves consistently with the standard NumPy approach, but integrates seamlessly into the QUEENS workflow with result tracking, logging, reproducibility, and generalisability.

___
### Generalisability of the QUEENS workflow

One of the main design ideas of QUEENS is: **set up your model once, then reuse it**.  
You can then run different multi-query analyses with the same model setup, gradually increasing the analysis complexity.

___

#### Example

- We already visualised the Rosenbrock function and saw its **banana-shaped valley**.  
- The valley looks flat in the plot, but the actual **minimum** is hidden inside it.  
- Finding this minimum is an **optimisation problem**.  

We can solve it in QUEENS by simply switching from a `Grid` iterator to an **`Optimization` iterator** — (almost) no need to change the model setup.
The only thing that we have to change in the model is the scheduler, since we need to request new compute resources, so we have to create a new scheduler; all other aspects of the model can stay the same, though!
Since this we are conducting a new experiment, we are also introducing a new global setting with a new experiment name.

One of the most important parameters specific to the `Optimization` iterator is the initial guess.

In [ ]:
from queens.iterators import Optimization

global_settings_optimization = GlobalSettings(experiment_name="optimization_rosenbrock", output_dir="./")

scheduler_optimization = Local(global_settings_optimization.experiment_name, num_jobs=1)
model_optimization = Simulation(scheduler_optimization, driver)


optimization = Optimization(
    algorithm="L-BFGS-B",
    initial_guess=[-2.0, 3.0],
    bounds=[float("-inf"), float("inf")],
    max_feval=1e4,
    objective_and_jacobian=True,
    model=model_optimization,
    parameters=parameters,
    global_settings=global_settings_optimization,
    result_description={"write_results": True},
)

with global_settings_optimization:
    # Actual analysis
    run_iterator(optimization, global_settings=global_settings_optimization)

    # Load results
    results = load_result(global_settings_optimization.result_file(".pickle"))

    optimal_x = results.x
    optimal_fun = results.fun


You have successfully identified the minimum of the Rosenbrock function with a gradient based optimisation algorithm:

In [ ]:
print(f"The minimum of the Rosenbrock function {optimal_fun} is at {optimal_x}.")

Let's visualise the minimum together with the surface plot.

In [ ]:
visualize_grid_and_surface(X1, X2, Z, min_point=(optimal_x[0], optimal_x[0], optimal_fun))

## Optional tasks: time for some individualisation
1. The resolution of the grid is relatively coarse.
Increase the resolution by increasing the `num_grid_points` in the grid design and repeat the QUEENS experiment.

1. You can adjust the bounds of the grid per variable via the keywords `lower_bound` and `upper_bound` of the Uniform parameter objects. 
Go ahead and see what happens if you change the bound.

1. We are executing the study on our local machine, so we are using the
`Local` scheduler. Nevertheless, you can also run the model evaluations in parallel by increasing `num_jobs`.
See how the time for the calculation of an experiment changes by increasing `num_jobs`. Warning: don't go beyond the maximum number of CPUs your machine has.

___

## Advanced optional task: Try another test function for optimisation

To see how flexible QUEENS is, let’s try a different **test function** from the [list of optimisation test functions](https://en.wikipedia.org/wiki/Test_functions_for_optimization).

### Steps

1. **Pick a function**  
   Choose any test function you like (e.g. Rastrigin, Ackley, Himmelblau, …).

1. **Write it in Python**  
   Implement the function with the same variable names (`x1`, `x2`) so that QUEENS can recognise them.

1. **Define the parameters**  
   Create a new `Parameters` object with `x1` and `x2` in the correct domain for the chosen function.

1. **Wrap it in a QUEENS model**  
   - Create a new `Function` driver from your Python function  
   - Wrap it into a `Simulation` model  

1. **Run the analysis**  
   - First, use a `Grid` iterator to **visualise the surface** of your new function.  
   - Then, switch to an `Optimisation` iterator to **search for the minimum**.  

With these steps, you can quickly test different benchmark functions without changing the overall QUEENS workflow.